In [ ]:
# disable UserWarning
import warnings
warnings.simplefilter('ignore', category=UserWarning)

import numpy as np
import talib as ta
import vectorbt as vbt
import pandas as pd
import mplfinance as mpf


Example

- Get data
- Check indicators for backtesting
- Backtest with different parameters
- Analyse the results

For regular indicators see https://ta-lib.github.io/ta-lib-python/doc_index.html

Start by getting the data.
First download data in nb_fetch_.ipynb.

In [ ]:


symbols = [
    "BTCUSDT",
    # "ETHUSDT",
    # "BNBUSDT",
    # "DOGEUSDT",
    # "LTCUSDT",
    #  "SOLUSDT",
]
symbol = symbols[0]

end = 'now UTC'
interval = '1w'

# see https://github.com/polakowo/vectorbt/issues/301
vbt.settings.array_wrapper['freq'] = '7d'
sl_stop = 0.1

data = {}
ohlc_dict = {}

for symbol in symbols:
    df = pd.read_csv('../pdata/' + symbol + '_' + interval + '.csv')
    df.rename(columns={'timestamp': 'Date'}, inplace=True)

    df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)

    closes = pd.Series(df['close'])
    highs = pd.Series(df['high'])
    lows = pd.Series(df['low'])
    opens = pd.Series(df['open'])
    volumes = pd.Series(df['volume'])
    ohlc_dict[symbol] = {
        "closes": closes,
        "highs": highs,
        "lows": lows,
        "opens": opens,
        "volumes": volumes,
    }

    data[symbol] = df

In [ ]:
df_symbol = data[symbol]

df_symbol['ema'] = ta.EMA(df_symbol['close'], timeperiod=21)
df_symbol['sma'] = ta.SMA(df_symbol['close'], timeperiod=21)
df_symbol['sma_slow'] = ta.SMA(df_symbol['close'], timeperiod=50)

# Calculate roc (Rate of Change)
# Multiply by 100 to express it as a percentage
df_symbol['roc'] = ta.ROC(df_symbol['close'], timeperiod=14) * 100
df_symbol['rsi'] = ta.RSI(df_symbol['close'], timeperiod=14)
df_symbol['rsi_ma'] = ta.SMA(df_symbol['rsi'], timeperiod=10)

df_symbol['high_prev'] = df['high'].shift(12)
df_symbol['dip'] = (df['low'] - df['high_prev']) / df['high_prev'] * 100
df_symbol['dip_rolling'] = df_symbol['dip'].rolling(14).mean()

# percentile_10 = df_symbol['dip'].dropna().quantile(0.2)
# df_symbol['Markers'] = df_symbol['dip'] <= percentile_10
# marker_locations = df_symbol['Markers'].astype(int)

df_symbol['plus_di'] = ta.PLUS_DI(df_symbol['high'], df_symbol['low'], df_symbol['close'], timeperiod=14)
df_symbol['minus_di'] = ta.MINUS_DI(df_symbol['high'], df_symbol['low'], df_symbol['close'], timeperiod=14)
df_symbol['adx'] = ta.ADX(df_symbol['high'], df_symbol['low'], df_symbol['close'], timeperiod=14)

df_symbol['obv'] =ta.OBV(df_symbol['close'], df_symbol['volume'])

df_symbol['atr'] = ta.ATR(df_symbol['high'], df_symbol['low'], df_symbol['close'], timeperiod=14)
df_symbol['atr_ma'] = ta.SMA(df_symbol['atr'], timeperiod=10)

macd, macd_signal, macd_hist = ta.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['macd'] = macd
df['macd_signal'] = macd_signal
df['macd_hist'] = macd_hist
df_symbol.dropna(inplace=True)

# Prepare additional plots for roc
add_plots = [
    # mpf.make_addplot(marker_locations, type='scatter', color='green', marker='.', markersize=1, panel=0),
    mpf.make_addplot(df_symbol['ema'], panel=0, ylabel='Ema', color='purple'),
    mpf.make_addplot(df_symbol['sma'], panel=0, ylabel='Sma', color='blue'),
    mpf.make_addplot(df_symbol['sma_slow'], panel=0, ylabel='Sma', color='black'),
    

    mpf.make_addplot(df_symbol['roc'], panel=2,
                     ylabel='roc 3 m', color='blue'),

    mpf.make_addplot(df_symbol['rsi'], panel=3, ylabel='rsi', color='green'),
    mpf.make_addplot(df_symbol['rsi_ma'], panel=3, ylabel='rsi_ma', color='red'),

    # mpf.make_addplot(df_symbol['dip'], panel=4, ylabel='Dip', color='blue'),
    # mpf.make_addplot(df_symbol['dip_rolling'], panel=4,
    #                  ylabel='Dip Rolling', color='red'),
    
    mpf.make_addplot(df_symbol['plus_di'], panel=4
                     , ylabel='PLUS_DI', color='green'),
    
    mpf.make_addplot(df_symbol['minus_di'], panel=4, ylabel='MINUS_DI', color='red'),
    
    mpf.make_addplot(df_symbol['adx'], panel=4, ylabel='ADX', color='black'),
    
    mpf.make_addplot(df_symbol['obv'], panel=5, ylabel='OBV', color='black'),
    
    mpf.make_addplot(df_symbol['atr'], panel=6, ylabel='ATR', color='black'),
    mpf.make_addplot(df_symbol['atr_ma'], panel=6, ylabel='ATR_ma', color='red'),
    
    mpf.make_addplot(df_symbol['macd'], panel=7, ylabel='MACD', color='blue'),
    mpf.make_addplot(df_symbol['macd_signal'], panel=7, ylabel='MACD Signal', color='red'),
    mpf.make_addplot(df_symbol['macd_hist'], panel=7, ylabel='MACD Hist', color='black',  type='bar'),
    # make macd_hist as bar
     

]

# Plot candlestick chart with Volume and roc
mpf.plot(
    df_symbol,
    type='candle',      # Candlestick chart
    volume=True,        # Include volume panel
    addplot=add_plots,  # Add plots
    title='Candlestick Chart with Volume and roc',
    style='yahoo',      # Chart style
    ylabel='Price',
    figsize=(12, 8)
)

## Analyses

For some strategies it might make sense to nalyse outliers of the indicators / metrics.

In [ ]:
# change bins to 10
df_symbol['rsi'].hist(bins=100)
print(df_symbol['rsi'].describe())
X = 0.05
percentile_X = df_symbol['rsi'].dropna().quantile(X)
print(f"{X}th Percentile: {percentile_X}")
count_below_percentile_10 = (df_symbol['rsi'] <= percentile_X).sum()

print(f"Count of occurrences below or equal to the {X}th Percentile: {count_below_percentile_10}")

## Simple long strategy idea

Entry: PLUS_DI crosses above ADX and ADX is growing
Exit: ADX srops growing
Stop loss: 10%

In [ ]:
# Prepare params
def params_long() -> dict:
    adx_p = np.arange(14, 42 + 2, step=1)
    plus_di_p = np.arange(14, 42 + 2, step=1)

    permutations = (
        1
        * len(adx_p)
        * len(plus_di_p)
    )

    print(f"Number of permutations params_long: {permutations}")
    return {
        "adx_p": adx_p,
        "plus_di_p": plus_di_p,
    }

In [ ]:
## Prepare signal function

def signals_breakout_long(
    closes,
    highs,
    lows,
    opens,
    adx_p,
    plus_di_p,
):
    signal = np.full(closes.shape, np.nan)
    adx = ta.ADX(highs, lows, closes, timeperiod=adx_p)
    plus_di = ta.PLUS_DI(highs, lows, closes, timeperiod=plus_di_p)

    for idx in range(len(closes)):
        if idx < 1:
            signal[idx] = 0
            continue

        buy = (
            adx[idx] > adx[idx-1]
            and plus_di[idx] > adx[idx]
        )

        exit_buy = (
            adx[idx] < adx[idx-1]
        )

        if buy:
            signal[idx] = 1
        elif exit_buy:
            signal[idx] = -2
        else:
            signal[idx] = 0

    return signal

In [ ]:
# Compute signals function
def compute_signals_long(args: tuple):
    symbol, ohlc_dict, params = args
    indicator = vbt.IndicatorFactory(
        class_name="Long",
        short_name="long",
        input_names=[
            "closes",
            "highs",
            "lows",
            "opens",
        ],
        param_names=[
            "adx_p",
            "plus_di_p",
        ],
        output_names=["value"]).from_apply_func(
        signals_breakout_long,
        adx_p=10,
        plus_di_p=10,
        to_2d=False
    )
    closes = ohlc_dict["closes"]
    highs = ohlc_dict["highs"]
    lows = ohlc_dict["lows"]
    opens = ohlc_dict["opens"]
    res = indicator.run(
        closes,
        highs,
        lows,
        opens,
        adx_p=params["adx_p"],
        plus_di_p=params["plus_di_p"],
        param_product=True
    )

    entries = res.value == 1
    exits = res.value == -2
    short_entries = res.value == -1
    short_exits = res.value == 2

    print(f"End computing signals for {symbol}")
    return symbol, entries, exits, short_entries, short_exits

In [ ]:
params = params_long()
symbol, entries, exits, short_entries, short_exits = compute_signals_long(
    (symbol, ohlc_dict[symbol], params))

In [ ]:

pf = vbt.Portfolio.from_signals(close=closes,
                                entries=entries,
                                exits=exits,
                                short_entries=short_entries,
                                short_exits=short_exits,
                                init_cash=1000,
                                fees=0.001,
                                sl_stop=sl_stop,
                                open=opens,
                                high=highs,
                                low=lows,
                                direction='longonly',
                                
                                )

In [ ]:
figr = pf.total_return().vbt.heatmap(
    x_level='long_adx_p', y_level='long_plus_di_p', symmetric=False,
    trace_kwargs=dict(colorbar=dict(title='Total return')))
figr.show()

In [ ]:
total_return = pf.total_return().to_frame()
win_rate = pf.trades.win_rate().to_frame()
trade_count = pf.trades.count().to_frame()
profit_factor = pf.trades.profit_factor().to_frame()
# max_drawdown = pf.max_drawdown().to_frame()
expectancy = pf.trades.expectancy().to_frame()
# merge total_return and win_rate
merged = pd.concat(
    [total_return,
     win_rate,
     trade_count,
     profit_factor,
     ],
    axis=1)
# df_e = merged.sort_values(by='expectancy', ascending=False).head(5)
# print(df_e.head(5))

df_pf = merged.sort_values(by='profit_factor', ascending=False).head(5)
print(df_pf.head(5))

df_wr = merged.sort_values(by='win_rate', ascending=False).head(5)
print(df_wr.head(5))

In [ ]:
# get index values for the best profit factor
idx_values = df_pf.index.values[0]
pf[idx_values].stats()

In [ ]:
# get first row from df_pf
idx = 0
print("#########################")
win_rate = pf.trades[df_pf.index[idx]].win_rate()
print(symbol, f"{round(win_rate*100, 2)}%")
pf.trades[df_pf.index[idx]].plots(settings=dict(plot_zones=False)).show_svg()
pf[df_pf.index[idx]].plot_cum_returns().show_svg()
print("#########################")

- What about sl 5 %
- test with other symbols